# Exploratory Analyses for Self Prediction Experiments

Input
- **base completions**: .csv from the base completions
- **self prediction completions**: .csv from the self-prediction completions

Output
- .csv with random strings that are hard to predict from external means


In [ ]:
from pathlib import Path
import subprocess
import sys

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import words
from scipy import stats

In [ ]:
from compliance_checks import check_compliance

In [ ]:
# Set the display option to None to show all content
pd.set_option('display.max_colwidth', None)

In [ ]:
# Run the git command to get the repository root directory
REPO_DIR = subprocess.check_output(["git", "rev-parse", "--show-toplevel"]).decode().strip()

print("Repository directory:", REPO_DIR)
sys.path.append(REPO_DIR)

In [ ]:
# Set the directory for the data
EXPDIR = Path(REPO_DIR) / "exp"

In [ ]:
# What is the experiment with the base completions we want to use?
BASE_EXP = "string_gen_35" # 🔵
SELF_PRED_EXP =  BASE_EXP + "_self_prediction" 
FILENAME = "data0.csv" 

In [ ]:
# load the data
b_df = pd.read_csv(EXPDIR / BASE_EXP / FILENAME)
s_df = pd.read_csv(EXPDIR / SELF_PRED_EXP / FILENAME)
print(f"[Base] Loaded {len(b_df)} rows from {EXPDIR / SELF_PRED_EXP / FILENAME}")
print(f"[Self] Loaded {len(s_df)} rows from {EXPDIR / SELF_PRED_EXP / FILENAME}")

In [ ]:
# add in first logprobs
b_df["first_logprobs"] = b_df["logprobs"].apply(lambda x: eval(x)[0])
s_df["first_logprobs"] = s_df["logprobs"].apply(lambda x: eval(x)[0])

In [ ]:
# extract first token
def extract_top_token(logprobs):
    logprobs = eval(logprobs)
    top_token = list(logprobs[0].keys())[0]
    return top_token
    
b_df["first_token"] = b_df["logprobs"].apply(extract_top_token)
s_df["first_token"] = s_df["logprobs"].apply(extract_top_token)

In [ ]:
# make sure that the strings in s_df are all in b_df
assert set(s_df['string'].unique()).issubset(set(b_df['string'].unique())), "Not all strings in s_df are in b_df"

In [ ]:
dfs = {"Base": b_df, "Self": s_df}

In [ ]:
s_df.sample(3)

Run compliance checks. See `evals/compliance_checks.py` for more details.

In [ ]:
# The models like to repeat the last word. We flag it, but don't exclude it
def last_word_repeated(row):
    try:
        last_word = row['string'].split()[-1]
        return last_word.lower() == row['response'].lower()
    except AttributeError:
        return False

for name, df in dfs.items():
    df['last_word_repeated'] = df.apply(last_word_repeated, axis=1)
    print(f"[{name}] {df['last_word_repeated'].mean():.2%} of the responses repeat the last word")

In [ ]:
# Even if they don't repeat the last word, they like to repeat another word
def nonlast_word_repeated(row):
    try:
        nonlast_words = row['string'].split()[0:-1]
        nonlast_words = [w.lower() for w in nonlast_words]
        return row['response'].lower() in nonlast_words
    except AttributeError:
        return False

for name, df in dfs.items():
    df['nonlast_word_repeated'] = df.apply(nonlast_word_repeated, axis=1)
    print(f"[{name}] {df['nonlast_word_repeated'].mean():.2%} of the responses repeat a word other than the last word")

In [ ]:
for name, df in dfs.items():
    df['compliance'] = df['response'].apply(check_compliance)

In [ ]:
for name, df in dfs.items():
    print(f"[{name}] Compliance: {(df['compliance'] == True).mean():.2%}")

In [ ]:
for name, df in dfs.items():
    print(f"[{name}] Most common non-compliant reasons:")
    display(df[df['compliance'] != True]['compliance'].value_counts().head(10))

In [ ]:
# Sample some non-compliant responses
for name, df in dfs.items():
    print(f"[{name}] Sample non-compliant responses:")
    try:
        print(df[df['compliance'] != True].sample(3)[['string','response']])
    except ValueError:
        print("Not enough non-compliant responses to sample")
    finally:
        print()

In [ ]:
# Exclude non-compliant responses
for name in dfs.keys():
    dfs[name].query('compliance == True', inplace=True)
    print(f"[{name}] Excluded non-compliant responses, leaving {len(dfs[name])} rows")

In [ ]:
# we need to subset the data to only include the strings that are in both dfs
old_b_len = len(b_df)
strings_set = set(b_df['string'].unique()).intersection(set(s_df['string'].unique()))
b_df = b_df[b_df['string'].isin(strings_set)]
s_df = s_df[s_df['string'].isin(strings_set)]
print(f"Subsetted data to only include strings that are in both dfs.\nBefore: [Base] {old_b_len}, After: {len(b_df)}")

In [ ]:
# join the two dataframes on the string column
df = pd.merge(b_df, s_df, on="string", suffixes=("_base", "_self"))
df.drop(columns=["compliance_base", "compliance_self", "complete_base", "complete_self", "id_base", "id_self"], inplace=True)

In [ ]:
df

How many strings are correctly produced by the model?

In [ ]:
N_POSSIBLE_ITEMS = len(words.words()) # what is the number of possible items in the string?
print(f"Number of possible items in the string: {N_POSSIBLE_ITEMS},\nwhich gives us a probability of {1/N_POSSIBLE_ITEMS:.6%} for a random guess")

In [ ]:
rate_perfect_answer = (df['response_self'] == df['response_base']).mean()
print(f"Rate of perfect answer matches: {rate_perfect_answer:.2%}")
# t test
t, p = stats.ttest_1samp(df['response_self'] == df['response_base'], 1/N_POSSIBLE_ITEMS)
print(f"t = {t:.2f}, p = {p:.2f}")

In [ ]:
rate_first_token_match = (df['first_token_self'] == df['first_token_base']).mean()
print(f"Rate of first token matches: {rate_first_token_match:.2%}")
# t test
t, p = stats.ttest_1samp(df['first_token_self'] == df['first_token_base'], 1/N_POSSIBLE_ITEMS)
print(f"t = {t:.2f}, p = {p:.2f}")

In [ ]:
# let's qualitatively look at the matches
df[df['response_self'] == df['response_base']][['string', 'response_self', 'response_base', 'first_logprobs_base', 'last_word_repeated_self', 'nonlast_word_repeated_self']]

The models like to repeat the last word of the string. That is not a problem per se since we select the strings to be hard to predict.

In [ ]:
prop_repeat_last_word = df[df['response_self'] == df['response_base']]['last_word_repeated_self'].mean()
print(f"Proportion of response-matching strings where the model repeats the last word:\t{prop_repeat_last_word:.2%}")

In [ ]:
# if we exclude the strings where the model repeats the last word, what is the rate of perfect matches?
df_no_repeat = df[df['last_word_repeated_self'] == False]
print(f"There are {len(df_no_repeat)} rows where the model does not repeat the last word")
rate_perfect_answer_no_repeat = (df_no_repeat['response_self'] == df_no_repeat['response_base']).mean()
print(f"Rate of perfect answer matches (without word repetition):\t{rate_perfect_answer_no_repeat:.2%}")
# t test
t, p = stats.ttest_1samp(df_no_repeat['response_self'] == df_no_repeat['response_base'], 1/N_POSSIBLE_ITEMS)
print(f"t = {t:.2f}, p = {p:.2f}")

In [ ]:
# let's qualitatively look at the matches
df_no_repeat[df_no_repeat['response_self'] == df_no_repeat['response_base']][['string', 'response_self', 'response_base', 'last_word_repeated_self', 'nonlast_word_repeated_self']]

In [ ]:
# ok, what about those that aren't in the string at all?
df_no_repeat_no_repeats = df_no_repeat[df_no_repeat['nonlast_word_repeated_self'] == False]
print(f"There are {len(df_no_repeat_no_repeats)} rows where the model doesn't repeat the last word or any other word")
rate_perfect_answer_no_repeat_no_repeats = (df_no_repeat_no_repeats['response_self'] == df_no_repeat_no_repeats['response_base']).mean()
print(f"Rate of perfect answer matches (without word repetition and nonlast word repetition):\t{rate_perfect_answer_no_repeat_no_repeats:.2%}")
# t test
t, p = stats.ttest_1samp(df_no_repeat_no_repeats['response_self'] == df_no_repeat_no_repeats['response_base'], 1/N_POSSIBLE_ITEMS)
print(f"t = {t:.2f}, p = {p:.2f}")


In [ ]:
# let's look at this most selective set of strings
df_no_repeat_no_repeats[df_no_repeat_no_repeats['response_self'] == df_no_repeat_no_repeats['response_base']][['string', 'response_self', 'response_base', 'last_word_repeated_self', 'nonlast_word_repeated_self']]